In [6]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from sklearn.utils import resample
from IPython.display import display
import numpy as np
import datetime
import os

# Create a connection with SQL server to get data.
def exec_sql_query(query, param=None):
    
    from sqlalchemy import create_engine
    import urllib
    params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=LAPTOP-C3LFVOFI;DATABASE=student_life;UID=student_sense;PWD=abhinav123")
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    connection = engine.raw_connection()
    
    try:
        cursor = connection.cursor()
        if(param):
            cursor.execute(query, param)
        else : 
            cursor.execute(query)
            
        results = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame.from_records(results, columns=columns)
        cursor.close()
        connection.commit()
    finally:
        connection.close()
    
    del engine
    return df

In [7]:
def get_feature_query_for_student(dictionary, student_id):

    for key in dictionary.keys():
        dictionary[key] = dictionary[key]  + " where student_id = "+str(student_id) 
    
    return dictionary

In [8]:

# Maintaining a feature list.
base_feature_map = {

    "activity_details" : "SELECT activity_time ,student_id ,activity_inference FROM activity_details ",
    
    "dinning_details" : "SELECT dinning_time ,student_id ,venue_id ,meal_type_id FROM dinning_details  ",
    
    "call_log_details" : "select timestamp as call_time, student_id, 1 as call_recorded  from call_log_details",
    
    "sms_details" : "select timestamp, student_id, 1 from sms_details ",
    
    "audio_details" : "select audio_activity_time, student_id, audio_activity_inference from audio_details ",
    
    "conversation_details" : "select conv_start_timestamp, student_id, conv_duration_min from conversation_details ",
    
    "dark_details" : "select dark_start_timestamp, student_id, dark_duration_min from dark_details ",
    
    "phonecharge_details" : "select start_timestamp, student_id, phonecharge_duration_min from phonecharge_details ",

    "phonelock_details" : "select start_timestamp, student_id, phonelock_duration_min from phonelock_details ",
     
    "gps_details" : "select wifi_timestamp as time, student_id, latitude, longitude from gps_details "
    
                         }
base_feature_map = {
    
    "sleep_details" : "select response_timestamp, student_id, hours_slept, sleep_rating from sleep_details"
    
}
# base_feature_map = {
#     "dinning_details" : "SELECT dinning_time ,student_id ,venue_id ,meal_type_id FROM dinning_details  "
#     }

# Collecting distinct students.
distinct_students = exec_sql_query("select distinct student_id from stress_details")
distinct_students = distinct_students.values.T.tolist()
distinct_students = distinct_students[0]
distinct_students.sort()

# getting current working directory for creating directories later.
cwd = os.getcwd()

# distinct_students = distinct_students[10:20]
print("Students: ", distinct_students)



Students:  [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 27, 30, 31, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59]


In [9]:
for student in distinct_students:
    newpath = r'\StudentLife Data\student '+ str(student) 
    newpath = cwd + newpath

    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    # Getting Stress levels for only student_id = 1. This will be merged with other features.
    stress_details_raw = exec_sql_query("select student_id, response_time, stress_level from stress_details where student_id = "+str(student))
    stress_details = stress_details_raw.loc[:,["response_time","student_id","stress_level"]]
    stress_details = stress_details.sort_values(by="response_time")
    stress_details.rename({"response_time": "time"}, axis='columns', inplace=True)

    # Extracting first and last index of stress level. 
    # We will truncate other features 3 day behind and 3 day ahead.
    first_date = stress_details.loc[0, 'time']
    last_date = stress_details.loc[len(stress_details)-1, 'time']
    

    # delta to back and ahead, in days.
    first_date = first_date - datetime.timedelta(days=3)
    last_date = last_date + datetime.timedelta(days=0)
    
    feature_map = get_feature_query_for_student(base_feature_map.copy(), student)
    
    for key in feature_map.keys():

        feature_query = feature_map[key]
        
#         print(feature_query)
        # Data processing begins..
        feature_data = exec_sql_query(feature_query)

        # Selecting Time Col.
        train_col_list = []

        for col in feature_data.columns:
            if "time" in col:
                time_column = col
            else:
                train_col_list.append(col)

        feature_data.rename({time_column: "time"}, axis='columns', inplace=True)
        time_column = "time"

        # Sorting by values of time.        
        feature_data = feature_data.sort_values(by=time_column)
        feature_data = feature_data.dropna()

        # Truncating extra features that do not lie in the time frame.
        feature_data = feature_data[np.logical_and(feature_data[time_column] > first_date, feature_data[time_column] < last_date)]
                
        if feature_data.empty:
            print ("Empty Dataframe for Student {} for feature {}".format(student, key))
            continue
    
        
        train_feature_data = pd.merge_asof(stress_details,
                                     feature_data,
                                     direction="backward",
                                     by="student_id",
                                     on = "time")    
        
        # We need to append the other feature values as well. Creating New column for activity_details/
        feature_data["stress_level"] = np.nan 

        # Sorting column naming convention
        train_col_list = []
        
        for col in train_feature_data.columns:
            if col !="stress_level":
                train_col_list.append(col)

        train_col_list.append('stress_level')
        train_feature_data = train_feature_data[train_col_list]
        
        # Rearrainging columns
        feature_data = feature_data[train_col_list]
        train_feature_data = train_feature_data[train_col_list]
        
        train_feature_data = train_feature_data.append(feature_data, ignore_index=True)
        train_feature_data.sort_values(by=["student_id","time"], inplace=True)
        train_feature_data.reset_index(inplace=True, drop=True)
        
        # Filling any nulls in the feature data, but not touching the stress levels.
        train_feature_data.iloc[:,1:-1] = train_feature_data.iloc[:,1:-1].fillna(method="backfill", axis=0)
        train_feature_x = train_feature_data
        df_filter = train_feature_x.loc[:,["stress_level"]].stress_level.notnull()
        train_feature_y_indices = train_feature_x[df_filter].index
        train_feature_y_indices = pd.DataFrame(train_feature_y_indices)
        train_feature_y_indices.rename({0:"indices"}, axis=1, inplace=True)  
        
        # Saving in CSV.        
        train_feature_x.to_csv("StudentLife Data/student "+str(student)+"/"+key+"_train_x.csv", index=True)
        train_feature_y_indices.to_csv("StudentLife Data/student "+str(student)+"/"+key+"_train_y_indices.csv", index=True)   

Empty Dataframe for Student 13 for feature sleep_details
